In [1]:
import numpy as np

In [9]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [20]:
!wget https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/tf_keras_model.py

--2022-06-15 14:24:30--  https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/tf_keras_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7649 (7.5K) [text/plain]
Saving to: ‘tf_keras_model.py.2’

tf_keras_model.py.2 100%[===================>]   7.47K  --.-KB/s    in 0s      

2022-06-15 14:24:30 (94.7 MB/s) - ‘tf_keras_model.py.2’ saved [7649/7649]



In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# VANNO TUTTI CAMBIATI

# !wget https://www.dropbox.com/s/6s5khncrval4rz9/bkg_et.npy?dl=1 -o bkg_et.npy
# !wget https://www.dropbox.com/s/qc7z1nhqh5xvpbg/bkg_mask.npy?dl=1 -o bkg_mask.npy
# !wget https://www.dropbox.com/s/fs5e9wuhqtr66xh/bkg_xy.npy?dl=1 -o bkg_xy.npy
# !wget https://www.dropbox.com/s/w0b94xzyk2wg7y4/signal_et.npy?dl=1 -o signal_et.npy
# !wget https://www.dropbox.com/s/3k6b0u3zz65xnyf/signal_mask.npy?dl=1 -o signal_mask.npy
# !wget https://www.dropbox.com/s/chcep3q8terrj4k/signal_xy.npy?dl=1 -o signal_xy.npy

In [21]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite, edge_conv

In [17]:
data_folder = "/content/drive/MyDrive/datiML"

name_map = {"points": "xy", "features": "et", "mask": "mask"} # va provato sia xy che rphi


signal  = {key: np.load(f"{data_folder}/signal_{ name_map[key]}.npy") for key in name_map}

mnbs    = {key: np.load(f"{data_folder}/mnbs_{   name_map[key]}.npy") for key in name_map}

cosmics = {key: np.load(f"{data_folder}/cosmics_{name_map[key]}.npy") for key in name_map}

In [28]:
S =  signal["mask"].shape[0]
F =    mnbs["mask"].shape[0]
C = cosmics["mask"].shape[0]

N = S + F + C

p = np.random.permutation(N)

In [26]:
'''
cluster su disco 0 sopra 50 MeV in un injection cycle
RMC: 66
MNBS: 107k
cosmici: 4

cluster simulati su disco 0 sopra i 50 MeV:
RMC: 150k
MNBS: 16505 * 5 = 82k
cosmici: 53k

pesi:
RMC: 66/150k 
MNBS: 107k/82k
Cosmici: 4/53k
'''

signal_weight = 66/S * 100
mnbs_weight = 1.07e3/F * 100
cosmics_weight = 4/C * 100

In [32]:
data = {key: np.concatenate( (signal[key], mnbs[key], cosmics[key]) )[p] for key in name_map}

label_list = [1, 0, 0]
entries_list = [S, F, C]
weight_list = [signal_weight, mnbs_weight, cosmics_weight]

print(entries_list)
print(weight_list)

label = np.concatenate( 
    [ np.ones((e))*l for e, l in zip(entries_list,  label_list) ]
)[p]

sample_weight = np.concatenate( 
    [ np.ones((e))*w for e, w in zip(entries_list, weight_list) ]
)[p]

[63721, 82213, 52984]
[0.10357652893080774, 1.3014973301059445, 0.007549448890231013]


In [33]:
train_data  = {key: data[key][int(N*0.0) : int(N*0.7)] for key in name_map}
test_data   = {key: data[key][int(N*0.7) : int(N*0.9)] for key in name_map}
val_data    = {key: data[key][int(N*0.9) : int(N*1.0)] for key in name_map}

train_label = label[int(N*0.0) : int(N*0.7)]
test_label  = label[int(N*0.7) : int(N*0.9)]
val_label   = label[int(N*0.9) : int(N*1.0)]

train_weight = sample_weight[int(N*0.0) : int(N*0.7)]
test_weight  = sample_weight[int(N*0.7) : int(N*0.9)]
val_weight   = sample_weight[int(N*0.9) : int(N*1.0)]

In [34]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
num_classes = 2
input_shapes = {'points': (50, 2), 'features': (50, 2), 'mask':(50, 1)}

if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

In [35]:
# Training parameters
batch_size = 1024 if 'lite' in model_type else 384
epochs = 30

In [36]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [37]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
model.summary()

[2022-06-15 14:27:07,250] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 50, 1)]      0           []                               
                                                                                                  
 tf.math.not_equal (TFOpLambda)  (None, 50, 1)       0           ['mask[0][0]']                   
                                                                                                  
 tf.cast (TFOpLambda)           (None, 50, 1)        0           ['tf.math.not_equal[0][0]']      
                                                                                                  
 tf.math.equal (TFOpLambda)     (None, 50, 1)        0           ['tf.cast[0][0]']                
                                                                                        

In [38]:
# Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, progress_bar]

In [39]:
model.fit(
    [train_data["points"], train_data["features"], train_data["mask"]], 
    train_label,
    batch_size=batch_size,
    epochs=10,
    validation_data=([val_data["points"], val_data["features"], val_data["mask"]], val_label, val_weight),
    shuffle=True,
    sample_weight=train_weight
    #callbacks=callbacks
)

Epoch 1/10
136/136 [==============================] - 49s 249ms/step - loss: 0.1073 - accuracy: 0.7057 - precision: 0.7492 - recall: 0.1211 - val_loss: 0.2291 - val_accuracy: 0.7441 - val_precision: 0.7795 - val_recall: 0.2956
Epoch 2/10
136/136 [==============================] - 29s 215ms/step - loss: 0.0754 - accuracy: 0.7774 - precision: 0.8449 - recall: 0.3733 - val_loss: 0.1653 - val_accuracy: 0.8068 - val_precision: 0.7156 - val_recall: 0.6722
Epoch 3/10
136/136 [==============================] - 30s 219ms/step - loss: 0.0715 - accuracy: 0.7949 - precision: 0.8775 - recall: 0.4176 - val_loss: 0.1041 - val_accuracy: 0.8267 - val_precision: 0.8307 - val_recall: 0.5857
Epoch 4/10
136/136 [==============================] - 30s 224ms/step - loss: 0.0699 - accuracy: 0.8026 - precision: 0.8902 - recall: 0.4374 - val_loss: 0.1211 - val_accuracy: 0.8409 - val_precision: 0.8045 - val_recall: 0.6736
Epoch 5/10
136/136 [==============================] - 31s 225ms/step - loss: 0.0687 - accura